In [98]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [99]:
import numpy as np
import tflearn
import tensorflow as tsflw
import random as rd
import json

In [100]:
with open("intents bot1.json") as f:
	dat = json.load(f)
	intents = dat['intents']
intents

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is tim',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': ['I am 18 years old!', '18 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Tim.',
   "I'm Tim!",
   "I'm Tim aka Tech With Tim."],
  'context_set': ''},
 {'tag': 'shop',
  'patterns': ['Id like to buy something',
   'whats on the menu',
   'what do you reccommend?',
   'could i get som

In [101]:
words = []
labels = []
docs_x = []
docs_y = []
for intent in intents:
	for pattern in intent['patterns']:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])
	
	if intent["tag"] not in labels:
		labels.append(intent['tag'])
		

In [102]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"] # stemmer: extract root word 
words = sorted(list(set(words))) # remove duplicates
labels = sorted(labels)

In [103]:
'''
freqeuncy representation
[1,0,3] - freq
['a','','hi'] - corresponding word contained in the sent
'''
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

In [104]:
training = np.array(training)
output = np.array(output)

In [105]:
# Buidling AI
tsflw.compat.v1.reset_default_graph() # this is required to run afterwards
net = tflearn.input_data(shape=[None, len(training[0])])

net = tflearn.fully_connected(net, 8) # 8 neurons
net = tflearn.fully_connected(net, 8) # 8 neurons
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") # output layers
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [106]:
training.shape

(26, 46)

In [107]:
output.shape

(26, 6)

In [108]:
model.fit(training, output, n_epoch=1000, batch_size = 8, show_metric = True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.21352 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.21352 - acc: 0.9817 -- iter: 24/26
Training Step: 4000  | total loss: 0.20115 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.20115 - acc: 0.9836 -- iter: 26/26
--
INFO:tensorflow:c:\Users\Leong Teng Man\Documents\GitHub\AI-School-Project\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:c:\Users\Leong Teng Man\Documents\GitHub\AI-School-Project\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:c:\Users\Leong Teng Man\Documents\GitHub\AI-School-Project\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:c:\Users\Leong Teng Man\Documents\GitHub\AI-School-Project\model.tflearn.meta
INFO:tensorflow:100


In [113]:
def bag_of_words(sent, words):
	bag = [0 for _ in range(len(words))]
	sent_wrds = nltk.word_tokenize(sent)
	sent_wrds = [stemmer.stem(w.lower()) for w in sent_wrds]

	# counting freq for each word
	for word in sent_wrds:
		for i, w in enumerate(words):
			if w == word:
				bag[i] = 1
	return np.array(bag)


In [122]:
def chat():
	print("You're talking with the bot (type quit to quit)!")

	while True:
		user_inp = input("You: ")
		if user_inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(user_inp, words)])[0]
		results_idx = np.argmax(results)
		tag = labels[results_idx]
		if results[results_idx] > 0.7:

			for corspd in dat['intents']:
				if corspd['tag'] == tag:
					responses = corspd['responses']
			
			print(rd.choice(responses))
		else:
			print("I didn't get that, try again")
chat()

You're talking with the bot (type bye to quit)!
[2.8298786e-03 2.5551571e-02 9.0004575e-01 7.0179611e-02 3.5150442e-04
 1.0417529e-03]
2
Hello!
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.00564025]
2
I didn't get that, try again
[0.20513606 0.17796925 0.54831314 0.03820605 0.02473528 0.005640